In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
sb.set() # set the default Seaborn style for graphics
from pandas import DataFrame

excel_file = 'AviationData.csv'
AviationData = pd.read_csv(excel_file, sep=',', header=0, encoding = 'iso-8859-1')
AviationData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84262 entries, 0 to 84261
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                84262 non-null  object 
 1   Investigation.Type      84257 non-null  object 
 2   Accident.Number         84262 non-null  object 
 3   Event.Date              84262 non-null  object 
 4   Location                84185 non-null  object 
 5   Country                 83755 non-null  object 
 6   Latitude                30163 non-null  float64
 7   Longitude               30154 non-null  float64
 8   Airport.Code            47723 non-null  object 
 9   Airport.Name            50439 non-null  object 
 10  Injury.Severity         84262 non-null  object 
 11  Aircraft.Damage         81553 non-null  object 
 12  Aircraft.Category       27508 non-null  object 
 13  Registration.Number     80412 non-null  object 
 14  Make                    84192 non-null

## General Data Cleaning

In [2]:
AviationData

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.Flight,Air.Carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.Phase.of.Flight,Report.Status,Publication.Date
0,20200102X82407,Accident,WPR20CA055,31/12/2019,"Elk, CA",United States,39.128611,-123.715833,LLR,Little River,...,Personal,NaN,NaN,NaN,NaN,1.0,VMC,TAKEOFF,Factual,13/1/2020
1,20191231X83852,Accident,CEN20FA049,31/12/2019,"OLATHE, KS",United States,38.846111,-94.736111,OJC,Johnson County Executive,...,Personal,NaN,2.0,NaN,NaN,NaN,VMC,TAKEOFF,Preliminary,8/1/2020
2,20200102X54844,Accident,ANC20CA011,31/12/2019,"Fairbanks, AK",United States,64.666945,-148.133334,NaN,NaN,...,Personal,NaN,NaN,NaN,NaN,2.0,NaN,NaN,Preliminary,2/1/2020
3,20191230X91852,Accident,CEN20CA048,30/12/2019,"GRANBURY, TX",United States,32.365556,-97.645000,NaN,NaN,...,Personal,NaN,NaN,NaN,NaN,1.0,NaN,NaN,Preliminary,31/12/2019
4,20191228X62945,NaN,WPR20CA053,28/12/2019,"Missoula, MT",United States,NaN,NaN,MSO,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Preliminary,3/1/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84257,20041105X01764,Accident,CHI79FA064,2/8/1979,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,NaN,VMC,APPROACH,Probable Cause,16/4/1980
84258,20001218X45448,Accident,LAX96LA321,19/6/1977,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,CRUISE,Probable Cause,12/9/2000
84259,20061025X01555,Accident,NYC07LA005,30/8/1974,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,CRUISE,Probable Cause,26/2/2007
84260,20001218X45447,Accident,LAX94LA336,19/7/1962,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,UNKNOWN,Probable Cause,19/9/1996


In [4]:
# Taking only the accidents occured in United States to make it more focused
CleanData = AviationData[AviationData['Country'] == "United States"]

# Dropping unnecessary columns
CleanData = CleanData.drop(columns=['Investigation.Type', 'Accident.Number', 'Location', 'Airport.Name', 'Injury.Severity', 'Registration.Number', 'FAR.Description', 'Schedule', 'Air.Carrier','Publication.Date', 'Report.Status', 'Publication.Date'])

# Dropping unknown aircraft damage because it is our response variable
CleanData = CleanData.dropna(subset=['Aircraft.Damage'])

# Generalizing the data by turning it to upper case
CleanData['Make'] = CleanData['Make'].str.upper()

# Creating new columns to extract month and year individually for further purposes
date = pd.DatetimeIndex(CleanData['Event.Date'])
CleanData['Month'] = date.month
CleanData['Year'] = date.year
CleanData['Time'] = pd.to_datetime(CleanData['Event.Date']).dt.to_period('M')
CleanData = CleanData[CleanData['Year'] >= 2000]

CleanData.head(50)

,Event.Id,Event.Date,Country,Latitude,Longitude,Airport.Code,Aircraft.Damage,Aircraft.Category,Make,Model,...,Purpose.of.Flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.Phase.of.Flight,Month,Year,Time
0,20200102X82407,31/12/2019,United States,39.128611,-123.715833,LLR,Substantial,Airplane,CESSNA,175,...,Personal,NaN,NaN,NaN,1.0,VMC,TAKEOFF,12,2019,2019-12
1,20191231X83852,31/12/2019,United States,38.846111,-94.736111,OJC,Destroyed,Airplane,MOONEY,M20S,...,Personal,2.0,NaN,NaN,NaN,VMC,TAKEOFF,12,2019,2019-12
2,20200102X54844,31/12/2019,United States,64.666945,-148.133334,NaN,Substantial,Airplane,CESSNA,170,...,Personal,NaN,NaN,NaN,2.0,NaN,NaN,12,2019,2019-12
3,20191230X91852,30/12/2019,United States,32.365556,-97.645000,NaN,Substantial,Airplane,VANS,RV 10,...,Personal,NaN,NaN,NaN,1.0,NaN,NaN,12,2019,2019-12
4,20191228X62945,28/12/2019,United States,NaN,NaN,MSO,Substantial,NaN,CESSNA,170,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,2019,2019-12
5,20191228X23853,28/12/2019,United States,30.176111,-92.007500,LFT,Destroyed,Airplane,PIPER,PA 31T,...,Executive/Corporate,5.0,2.0,2.0,NaN,IMC,TAKEOFF,12,2019,2019-12
6,20191225X93635,25/12/2019,United States,31.364167,-85.312500,0J6,Substantial,Helicopter,BELL,407,...,NaN,1.0,NaN,NaN,2.0,VMC,APPROACH,12,2019,2019-12
8,20191223X92250,23/12/2019,United States,NaN,NaN,CHD,Substantial,NaN,PIPER,PA28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,2019,2019-12
9,20191223X91929,22/12/2019,United States,34.440000,-114.345555,NaN,Substantial,NaN,COSMOS,Phase II,...,Personal,NaN,1.0,NaN,1.0,VMC,NaN,12,2019,2019-12
10,20191221X83621,21/12/2019,United States,38.095833,-87.540555,EVV,Destroyed,Airplane,PIPER,PA28,...,Personal,1.0,NaN,NaN,NaN,NaN,APPROACH,12,2019,2019-12


In [5]:
# filling empty rows with default values
values = {'Total.Fatal.Injuries': 0, 'Total.Serious.Injuries': 0, 'Total.Minor.Injuries': 0, 'Total.Uninjured': 0, 'Number.of.Engines': 0, 'Broad.Phase.of.Flight': "UNKNOWN", 'Airport.Code': "XXX", 'Weather.Condition': "UNK", 'Engine.Type': "Unknown", 'Purpose.of.Flight': "Unknown", 'Aircraft.Category' : "Unknown", 'Latitude': 0, 'Longitude': 0, 'Model' : "UNKNOWN", 'Make' : "UNKNOWN", 'Amateur.Built' : "Unknown"}
CleanData = CleanData.fillna(value=values)
CleanData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31506 entries, 0 to 36586
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype    
---  ------                  --------------  -----    
 0   Event.Id                31506 non-null  object   
 1   Event.Date              31506 non-null  object   
 2   Country                 31506 non-null  object   
 3   Latitude                31506 non-null  float64  
 4   Longitude               31506 non-null  float64  
 5   Airport.Code            31506 non-null  object   
 6   Aircraft.Damage         31506 non-null  object   
 7   Aircraft.Category       31506 non-null  object   
 8   Make                    31506 non-null  object   
 9   Model                   31506 non-null  object   
 10  Amateur.Built           31506 non-null  object   
 11  Number.of.Engines       31506 non-null  float64  
 12  Engine.Type             31506 non-null  object   
 13  Purpose.of.Flight       31506 non-null  object   
 14  Total.

## Assigning Severity Scores and Classes

In [6]:
# Creating a column of the number of total passengers and injury rate
CleanData['Total.Passengers'] = CleanData['Total.Fatal.Injuries'] + CleanData['Total.Serious.Injuries'] + CleanData['Total.Minor.Injuries'] + CleanData['Total.Uninjured']
CleanData = CleanData[CleanData['Total.Passengers'] != 0]
CleanData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31403 entries, 0 to 36586
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype    
---  ------                  --------------  -----    
 0   Event.Id                31403 non-null  object   
 1   Event.Date              31403 non-null  object   
 2   Country                 31403 non-null  object   
 3   Latitude                31403 non-null  float64  
 4   Longitude               31403 non-null  float64  
 5   Airport.Code            31403 non-null  object   
 6   Aircraft.Damage         31403 non-null  object   
 7   Aircraft.Category       31403 non-null  object   
 8   Make                    31403 non-null  object   
 9   Model                   31403 non-null  object   
 10  Amateur.Built           31403 non-null  object   
 11  Number.of.Engines       31403 non-null  float64  
 12  Engine.Type             31403 non-null  object   
 13  Purpose.of.Flight       31403 non-null  object   
 14  Total.

In [7]:
#Assigning values of 1,2, or 3 to 'Minor', 'Substantial', and 'Destroyed' labels in Aircraft Damage
CleanData["Num_Aircraft_Damage"] = CleanData["Aircraft.Damage"].replace({"Minor" : 0.8, "Substantial" : 0.9, "Destroyed" : 1, "Unknown" : 0.8})

In [8]:
# Assigning Severity Scores to the data
CleanData['Severity.Score'] = ((8.75*CleanData['Total.Fatal.Injuries'] + 4.25*CleanData['Total.Serious.Injuries'] + 1*CleanData['Total.Minor.Injuries'])/CleanData['Total.Passengers'] + 1.25) * CleanData["Num_Aircraft_Damage"]
CleanData

,Event.Id,Event.Date,Country,Latitude,Longitude,Airport.Code,Aircraft.Damage,Aircraft.Category,Make,Model,...,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.Phase.of.Flight,Month,Year,Time,Total.Passengers,Num_Aircraft_Damage,Severity.Score
0,20200102X82407,31/12/2019,United States,39.128611,-123.715833,LLR,Substantial,Airplane,CESSNA,175,...,0.0,1.0,VMC,TAKEOFF,12,2019,2019-12,1.0,0.9,1.125000
1,20191231X83852,31/12/2019,United States,38.846111,-94.736111,OJC,Destroyed,Airplane,MOONEY,M20S,...,0.0,0.0,VMC,TAKEOFF,12,2019,2019-12,2.0,1.0,10.000000
2,20200102X54844,31/12/2019,United States,64.666945,-148.133334,XXX,Substantial,Airplane,CESSNA,170,...,0.0,2.0,UNK,UNKNOWN,12,2019,2019-12,2.0,0.9,1.125000
3,20191230X91852,30/12/2019,United States,32.365556,-97.645000,XXX,Substantial,Airplane,VANS,RV 10,...,0.0,1.0,UNK,UNKNOWN,12,2019,2019-12,1.0,0.9,1.125000
5,20191228X23853,28/12/2019,United States,30.176111,-92.007500,LFT,Destroyed,Airplane,PIPER,PA 31T,...,2.0,0.0,IMC,TAKEOFF,12,2019,2019-12,9.0,1.0,7.277778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36581,20001212X20364,2/1/2000,United States,0.000000,0.000000,XXX,Substantial,Unknown,PIPER,PA-46-310P,...,0.0,5.0,VMC,CRUISE,2,2000,2000-02,5.0,0.9,1.125000
36582,20001212X20358,2/1/2000,United States,0.000000,0.000000,XXX,Substantial,Unknown,PIPER,PA-46-310P,...,0.0,2.0,VMC,CLIMB,2,2000,2000-02,2.0,0.9,1.125000
36583,20001212X20344,2/1/2000,United States,0.000000,0.000000,XXX,Destroyed,Unknown,CESSNA,421B,...,0.0,0.0,IMC,CLIMB,2,2000,2000-02,1.0,1.0,10.000000
36585,20001212X20407,1/1/2000,United States,0.000000,0.000000,XXX,Substantial,Unknown,CESSNA,550,...,0.0,3.0,VMC,CRUISE,1,2000,2000-01,3.0,0.9,1.125000


In [9]:
# Assigning Severity_Class to each row
bins = [0.9, 1.251, 9.01, np.inf] # Boundaries for each level
names = [1, 2, 3] # 3 levels of severity
CleanData = CleanData.assign(Severity_Class = pd.cut(CleanData['Severity.Score'], bins, labels=names))

In [10]:
CleanData['Severity_Class'].value_counts()

1    17629
2    11119
3     2655
Name: Severity_Class, dtype: int64

In [37]:
p = pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
p.quantile(.1)

1.9

In [9]:
#Writing the new dataframe back to csv for use in other files
CleanData.to_excel('cleandata_.xlsx')

### <font color=#800020>This classification results on over 50% of the data being allocated to class 1. This might sound bad, but there are over 30% data between the boundary of class 1 and class 2. Hence, lowering class 2's boundary will result in over 60% of the data allocated to class 2, which is more disastrous. As a result, we stick to our initial classification.</font>